In [7]:
pip install xlsxwriter



  Obtaining dependency information for xlsxwriter from https://files.pythonhosted.org/packages/f7/3e/05ba2194cd5073602422859c949a4f21310a3c49bf8dccde9e03d4522b11/XlsxWriter-3.1.9-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/154.8 kB ? eta -:--:--
   -- ------------------------------------- 10.2/154.8 kB ? eta -:--:--
   ---------- ---------------------------- 41.0/154.8 kB 495.5 kB/s eta 0:00:01
   ---------------------------------------  153.6/154.8 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 154.8/154.8 kB 1.0 MB/s eta 0:00:00


In [36]:
import pandas as pd

# Replace 'eye_tracking.xlsx' with the path to your Excel file
input_file_path = 'eye_tracking.xlsx'
output_file_path = 'new_file_with_headers_only.xlsx'

# Load the Excel file
xls = pd.ExcelFile(input_file_path)

# Create a new Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    # Loop through each sheet in the Excel file
    for sheet_name in xls.sheet_names:
        # Read the current sheet into a DataFrame
        df = pd.read_excel(xls, sheet_name=sheet_name)

        # Select the first two rows (header rows)
        header_rows = df.iloc[:1]

        # Write the header rows to the new Excel file, maintaining the same sheet name
        header_rows.to_excel(writer, sheet_name=sheet_name, index=False)

print("The first two rows from each sheet have been copied to a new Excel file.")


The first two rows from each sheet have been copied to a new Excel file.


In [39]:
import pandas as pd

# Replace with the path to your Excel file
input_file_path = 'eye_tracking_v2.xlsx'

# Load the Excel file
xls = pd.ExcelFile(input_file_path)

# Iterate through each sheet in the Excel file
for sheet_name in xls.sheet_names:
    # Read the current sheet into a DataFrame
    df = pd.read_excel(xls, sheet_name=sheet_name)

    # Initialize variables to keep track of the previous 'base' column index and group
    prev_base_col_idx = None
    group = 0

    # Iterate through the columns starting from the third column
    for col_idx, col_name in enumerate(df.columns[2:], start=2):
        # Check if the column name contains 'base'
        if 'base' in col_name.lower():
            # Increment the group number and update the previous 'base' column index
            group += 1
            prev_base_col_idx = col_idx
        elif 'learning' in col_name.lower() or 'solving' in col_name.lower():
            # Calculate the absolute difference using the previous 'base' column
            df[f'group{group}_{col_name}_diff'] = abs(df.iloc[:, col_idx] - df.iloc[:, prev_base_col_idx])

    # Now, write the modified DataFrame back to the Excel file, overwriting the existing sheet
    with pd.ExcelWriter(input_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("The Excel file has been updated with the absolute differences.")


error: Error -3 while decompressing data: invalid distance too far back

In [58]:
import pandas as pd

# Define the normalization function
def normalize_column(column):
    min_val = column.min()
    max_val = column.max()
    # Apply the normalization formula
    normalized = (column - min_val) / (max_val - min_val)
    return normalized

# Path to the uploaded Excel file
file_path = 'Final Experiment 2_all.xlsx'  # Placeholder path, replace with actual path

# Load all sheets from the Excel file into a dictionary of DataFrames
xls = pd.ExcelFile(file_path)
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# Apply normalization to every column in each sheet (except for non-numeric columns)
for sheet_name, sheet_df in sheets_dict.items():
    # Normalize each numeric column in the DataFrame
    for col in sheet_df.select_dtypes(include=['number']).columns:
        sheet_df[col] = normalize_column(sheet_df[col])
    # Update the dictionary with the normalized DataFrame
    sheets_dict[sheet_name] = sheet_df

# Save the normalized data into a new Excel file with '_normalized' added to the file name
new_file_path = file_path.replace('.xlsx', '_normalized.xlsx')

# Use ExcelWriter to write each DataFrame to a separate sheet in the new Excel file
with pd.ExcelWriter(new_file_path) as writer:
    for sheet_name, sheet_df in sheets_dict.items():
        sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)

# Return the path to the new file as confirmation
new_file_path


'Final Experiment 2_all_normalized.xlsx'